Наше приложение будет решать вымышленную задачу, которую можно сформулировать при помощи следующих условий.
Компания под названием XYZ производит два типа компьютеров: «тип 1» и «тип 2». Тип компьютера определяется его архитектурой.
Компьютеры фирмы XYZ могут выполнять ряд функций. На данный момент определены четыре функции: сервер DDNS (DDNS Server), сервер DNS (DNS Server), шлюз (Gateway) и маршрутизатор (Router).
Каждый компьютер перед выпуском проходит ряд тестов.
Тесты, выполняемые над каждым компьютером, зависят от его типа и выполняемой функции. На данный момент определены пять тестов: «тест 1», «тест 2», «тест 3», «тест 4» и «тест 5».
Для каждого проверяемого компьютера устанавливается предельный срок тестирования. Все тесты, соответствующие данному компьютеру, должны быть выполнены не позднее указанной даты. Сама дата зависит от тестов, выбранных для каждого конкретного компьютера.
Большая часть процесса выполнения тестов в компании XYZ автоматизирована при помощи внутреннего программного обеспечения, которое выбирает конкретный набор тестов и определяет дату тестирования на основе типа и функций компьютеров.
Правила 
На данный момент наборы тестов и даты их выполнения для конкретных типов компьютеров выбираются в соответствии со следующими бизнес-правилами.
Над компьютерами типа 1 должны быть выполнены тесты 1, 2 и 3.
Над компьютерами типа 2, выполняющими функцию серверов DNS, должны быть выполнены тесты 4 и 5.
Над компьютерами типа 2, выполняющими функцию серверов DDNS, должны быть выполнены тесты 2 и 3.
Над компьютерами типа 2, выполняющими функцию шлюза, должны быть выполнены тесты 3 и 4.
Над компьютерами типа 2, выполняющиими функцию маршрутизатора, должны быть выполнены тесты 1 и 3.
Если среди тестов, выбранных для данного компьютера, есть тест 1, то тестирование должно производиться не позднее чем через три дня после даты производства. Данное правило является приоритетным по отношению ко всем последующим правилам выбора даты тестирования.
Если среди тестов, выбранных для данного компьютера, есть тест 2, то тестирование должно производиться не позднее чем через семь дней после даты производства. Данное правило является приоритетным по отношению ко всем последующим правилам выбора даты тестирования.
Если среди тестов, выбранных для данного компьютера, есть тест 3, то тестирование должно производиться не позднее чем через 10 дней после даты производства. Данное правило является приоритетным по отношению ко всем последующим правилам выбора даты тестирования.
Если среди тестов, выбранных для данного компьютера, есть тест 4, то тестирование должно производиться не позднее чем через 12 дней после даты производства. Данное правило является приоритетным по отношению ко всем последующим правилам выбора даты тестирования.
Если среди тестов, выбранных для данного компьютера, есть тест 5, то тестирование должно производиться не позднее чем через 14 дней после даты производства.

In [1]:
!pip install frozendict --upgrade

In [3]:
!pip install experta

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3165 sha256=bf212b480bd5c8420ab3d76465ecf0ef2a2957e87d03938f9f07295c7f088855
  Stored in directory: c:\users\moham\appdata\local\pip\cache\wheels\5b\fa\ab\0a80360debb57b95f092356ee3a075bbbffc631b9813136599
Successfully built frozendict
  Attempting uninstall: frozendict
    Found existing installation: frozendict 2.4.6
    Uninstalling frozendict-2.4.6:
      Successfully uninstalled frozendict-2.4.6


In [4]:
from collections.abc import Mapping
from experta import *

In [5]:
!pip show frozendict

Name: frozendict
Version: 1.2
Summary: An immutable dictionary
Home-page: https://github.com/slezica/python-frozendict
Author: Santiago Lezica
Author-email: slezica89@gmail.com
License: MIT License
Location: c:\users\moham\appdata\local\programs\python\python39\lib\site-packages
Requires: 
Required-by: experta


In [6]:
class Computer(Fact):
    """Fact representing a computer."""
    type = Field(str, mandatory=True)  # "type 1" or "type 2"
    function = Field(str, default=None)  # Optional function
    production_date = Field(str, mandatory=True)  # Production date in ISO format

class TestSelection(KnowledgeEngine):
    @Rule(Computer(type='type 1'))
    def assign_type1_tests(self):
        self.declare(Fact(tests=[1, 2, 3]))

    @Rule(Computer(type='type 2', function='DNS Server'))
    def assign_type2_dns_tests(self):
        self.declare(Fact(tests=[4, 5]))

    @Rule(Computer(type='type 2', function='DDNS Server'))
    def assign_type2_ddns_tests(self):
        self.declare(Fact(tests=[2, 3]))

    @Rule(Computer(type='type 2', function='Gateway'))
    def assign_type2_gateway_tests(self):
        self.declare(Fact(tests=[3, 4]))

    @Rule(Computer(type='type 2', function='Router'))
    def assign_type2_router_tests(self):
        self.declare(Fact(tests=[1, 3]))

    @Rule(Fact(tests=MATCH.tests), Computer(production_date=MATCH.production_date))
    def calculate_deadline(self, tests, production_date):
        import datetime
        from dateutil.parser import parse

        # Parse the production date
        prod_date = parse(production_date)
        deadlines = {
            1: 3,
            2: 7,
            3: 10,
            4: 12,
            5: 14
        }

        # Calculate the earliest deadline
        max_days = min(deadlines[test] for test in tests)
        deadline = prod_date + datetime.timedelta(days=max_days)

        # Format and print the output
        tests_str = ", ".join(map(str, tests))  # Convert list to comma-separated string
        print(f"Tests: {tests_str}")
        print(f"Deadline: {deadline.date()}")


In [7]:
# Usage example
engine = TestSelection()

# Input a computer of type 2 with the function 'Router' produced on 2023-12-01
engine.reset()
engine.declare(Computer(type='type 1', production_date="2023-12-01"))
engine.run()

Tests: 1, 2, 3
Deadline: 2023-12-04


In [8]:
# Input a computer of type 2 with the function 'Router' produced on 2023-12-01
engine.reset()
engine.declare(Computer(type='type 2', function='DNS Server', production_date="2023-12-01"))
engine.run()

Tests: 4, 5
Deadline: 2023-12-13


In [9]:
# Input a computer of type 2 with the function 'Router' produced on 2023-12-01
engine.reset()
engine.declare(Computer(type='type 2', function='DDNS Server', production_date="2023-12-01"))
engine.run()

Tests: 2, 3
Deadline: 2023-12-08


In [10]:
# Input a computer of type 2 with the function 'Router' produced on 2023-12-01
engine.reset()
engine.declare(Computer(type='type 2', function='Gateway', production_date="2023-12-01"))
engine.run()

Tests: 3, 4
Deadline: 2023-12-11


In [11]:
# Input a computer of type 2 with the function 'Router' produced on 2023-12-01
engine.reset()
engine.declare(Computer(type='type 2', function='Router', production_date="2023-12-01"))
engine.run()

Tests: 1, 3
Deadline: 2023-12-04
